### Packages

In [1]:
import sys
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
from keras_metrics import KerasMetrics
from keras.callbacks import EarlyStopping
from keras import optimizers
import numpy as np
import random
import pickle
import decoding_ngrams as dc
from Files import nodes
import os
import utility as ut
import livelossplot
sys.path.append('../')

Using TensorFlow backend.


In [2]:
def pipeline(verbs):
    """
    Pipeline receives a list of verbs and predicts a conjugated form for them.
    The prediction is saved in an output file.

    :verbs type: list
    :r type: string
    """
    import coding_function as cf
    from time import gmtime, strftime
    import itertools
    time = strftime("%Y-%m-%d_%H:%M:%S", gmtime())
    name = "output_" + time + ".txt"

    correct = {'#pega#': '#pEgu#', '#sega#': '#sEgu#', '#seka#': '#sEku#',
               '#leva#': '#lEvu#', '#ora#': '#Oru#', '#mora#': '#mOru#',
               '#posta#': '#pOtu#', '#joga#': '#jOgu#', '#sortia#': '#soiu#',
               '#media#': '#meiu#', '#kompo#': '#koiu#', '#po#': '#poiu#', 
               '#menti#': '#mitu#', '#tosi#': '#tusu#', '#kobri#': 
               '#kuro#', '#faze#': '#fasu#', '#mata#': '#matu#', '#paga#': 
               '#pagu#', '#sai#': '#saiu#', '#bate#': '#batu#', '#kome#': 
               '#komu#','#nota#': '#nOta#','#soka#': '#sOku#','#loga#': '#lOga#',
               '#limpa#': '#lipu#','#kasa#': '#kasu#'}

#     f = open('Files/Results/'+ name, "w")
#     f.write("train data set:" + path + '\n' + 'Results:\n' + 'model name: ' + load + '\n')
    test_list = []
    for i in verbs:
        coding = cf.coding(i)
        test_list.append(coding)
    test_list = np.array(test_list)
    prediction = model.predict(test_list)
    sum = 0
    for i,j in list(zip(verbs, prediction)):
#         f.write('verb: '+ i + ", expected: " + correct[i] + ", prediction: " + 
#                 dc.decoding(j) + '\n')
        decoded = dc.decoding(j)
        try: 
            print('input: '+ i + ", expected: " + correct[i] + ", output: " + 
                  decoded + '\n')
            if decoded == correct[i]:
                sum = sum + 1 
        except: 
            print('input: '+ i + " -> output: " + decoded + '\n')
        
   
    print("Accuracy (verbs): %.2f%%" % (sum/len(verbs)*100 ))
    scores = model.evaluate(X, Y)
#     f.write("\n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%%" % (model.metrics_names[0], scores[1]*100,
#                                                         model.metrics_names[1], scores[1]*100, 
#                                                         model.metrics_names[2], scores[2]*100,
#                                                         model.metrics_names[3], scores[3]*100 ))
    print("\n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100,
                                                                   model.metrics_names[1], scores[1]*100, 
                                                                   model.metrics_names[2], scores[2]*100,
                                                                   model.metrics_names[3], scores[3]*100 ))
    
    
    #f.close()
    return #print("\nPredictions saved in the file: " + name) 

### 1. Load Data

In [3]:
path = 'Corpus/train_corpus.csv'
X,Y = ut.load_ngrams(path)
batch = len(X)
batch

325

#### 1.2 Verify length of unique verbs

In [9]:
path = 'Corpus/train_corpus.csv'
X,Y = ut.load_ngrams(path, verbose = True)

AttributeError: module 'utility' has no attribute 'load_ngrams'

### 2. Fit Neural Net (Optional)

In [65]:
# 1. Define Model
model = Sequential()
model.add(Dense(1060, input_shape=(1060,), activation='sigmoid'))


# 2. Compile model
keras_metrics = KerasMetrics()
model.compile( 
    optimizer='adam', 
    loss='mean_squared_error',
    metrics = [ keras_metrics.fbeta_score,
               keras_metrics.recall,
               keras_metrics.precision]
            )

# 3. Fit model
stopper = EarlyStopping(monitor='fbeta_score', min_delta=0.00005, patience=50, verbose=1, mode='max')
model.fit(X,Y,epochs = 400, batch_size=batch,verbose=False,callbacks= [stopper])

# 4. Evaluate model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100,
                                                               model.metrics_names[1], scores[1]*100, 
                                                               model.metrics_names[2], scores[2]*100,
                                                               model.metrics_names[3], scores[3]*100 ))

# 5. Save model
model.save('Models/NgramModels/second_test')

256/325 [======================>.......] - ETA: 0s
loss: 2.71% 
fbeta_score: 92.49% 
recall: 86.15% 
precision: 100.00%


2. Load Trained Model

In [66]:
keras_metrics = KerasMetrics()
model = load_model('Models/NgramModels/second_test',custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})

#### 2.1 Train existing model

model.fit(X,Y,epochs = 150, batch_size=batch,verbose=False,callbacks= [stopper])

### 3. Check Results
Test the prediction for some verbs.

pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#tendi#','#jenti#',
          '#menti#','#hendi#','#tosi#','#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])
    

|       	|       	| front 	| middle  	| back    	|
|:-----:	|-------	|------:	|---------	|---------	|
|       	|       	| v/u   	| v/u     	| v/u     	|
|  **int**  | **stop**  | b/p   	| d/t     	| g/k     	|
|       	| **nasal** | m/-   	| n/-     	| -       	|
| **cont**  | **fric**  | v/f   	| z/s    	| j/x   	|
|       	| **liq**   | l/-   	| r/-     	| -/h     	|
| **vowel** | **high**  | e/i   	| -       	| o/u     	|
|       	| **low**   | -/E   	| -/a     	| -/O     	|

## Results:

In [6]:
from ngrams_nodes import activation

In [75]:
verbs = ['#mata#']
test_list = []
for i in verbs:
    coding = activation(i)
    test_list.append(coding)
test_list = np.array(test_list)
prediction = model.predict(test_list)

In [76]:
prediction

array([[ 0.29995853,  0.33579236,  0.31932685, ...,  0.32640621,
         0.30556184,  0.33054301]], dtype=float32)

In [77]:
from decoding_ngrams import decoding

In [78]:
for predict in prediction:
    print(decoding(predict))

       trigram     value
69   [#, m, a]  0.449999
589  [m, a, n]  0.490798
182  [a, n, t]  0.384592
671  [n, t, u]  0.360832
955  [t, u, #]  0.447232


In [11]:
len(prediction)

1